## Deliverable 2. Create a Customer Travel Destinations Map.

In [4]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [5]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Monrovia,LR,6.3005,-10.7969,75.25,75,70,6.73,broken clouds
1,1,Lompoc,US,34.6391,-120.4579,87.01,72,0,13.80,clear sky
2,2,Kuvshinovo,RU,57.0295,34.1725,48.45,88,18,3.40,few clouds
3,3,Dikson,RU,73.5069,80.5464,41.36,94,100,9.69,overcast clouds
4,4,Kruisfontein,ZA,-34.0033,24.7314,49.46,81,88,12.64,light rain


In [6]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [7]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Monrovia,LR,6.3005,-10.7969,75.25,75,70,6.73,broken clouds
1,1,Lompoc,US,34.6391,-120.4579,87.01,72,0,13.80,clear sky
7,7,Mayo,US,38.8876,-76.5119,84.15,73,0,3.44,clear sky
14,14,Iracoubo,GF,5.4802,-53.2011,77.88,86,96,8.70,overcast clouds
23,23,Georgetown,MY,5.4112,100.3354,82.89,93,20,3.44,few clouds
26,26,Fare,PF,-16.7000,-151.0167,78.98,70,0,22.82,clear sky
27,27,Coahuayana,MX,18.7333,-103.6833,81.39,79,100,1.81,overcast clouds
30,30,Vaini,TO,-21.2000,-175.2000,80.76,83,40,11.50,scattered clouds
31,31,San Cristobal,VE,7.7669,-72.2250,75.38,93,20,3.27,few clouds
36,36,Alcudia,ES,39.8532,3.1214,81.19,57,3,3.49,clear sky


In [8]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [9]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [10]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Monrovia,LR,75.25,broken clouds,6.3005,-10.7969,
1,Lompoc,US,87.01,clear sky,34.6391,-120.4579,
7,Mayo,US,84.15,clear sky,38.8876,-76.5119,
14,Iracoubo,GF,77.88,overcast clouds,5.4802,-53.2011,
23,Georgetown,MY,82.89,few clouds,5.4112,100.3354,
26,Fare,PF,78.98,clear sky,-16.7000,-151.0167,
27,Coahuayana,MX,81.39,overcast clouds,18.7333,-103.6833,
30,Vaini,TO,80.76,scattered clouds,-21.2000,-175.2000,
31,San Cristobal,VE,75.38,few clouds,7.7669,-72.2250,
36,Alcudia,ES,81.19,clear sky,39.8532,3.1214,


In [11]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")    


Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [12]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [13]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_Vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [14]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [15]:
# 11a. Add a marker layer for each city to the map. 
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

In [ ]:
# Take a screenshot of your map and save it to the Vacation_Search folder as WeatherPy_vacation_map.png.